In [9]:
import argparse
import os
import time
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.optim
from torch.nn.utils import clip_grad_norm_
from data import TrainStation
from motsynth import MOTSynth, MOTSynthBlackBG
from log_utils import log_summary
from utils import save_ckpt, load_ckpt, print_scalor
from common import *
import parse
from utils import spatial_transform, visualize
from tensorboardX import SummaryWriter
from scalor import SCALOR
from pycocotools import mask as coco_mask

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def convert_coco_poly_to_mask(segmentations, height, width):
    masks = []
    for polygons in segmentations:
        if isinstance(polygons['counts'], list):
            rles = coco_mask.frPyObjects(polygons, height, width)
        
        else:
            rles = [polygons]

        mask = coco_mask.decode(rles)
        if len(mask.shape) < 3:
            mask = mask[..., None]
        #mask = torch.as_tensor(mask, dtype=torch.uint8)
        #mask = mask.any(dim=2)
        masks.append(mask)
    # if masks:
    #     masks = torch.stack(masks, dim=0)
    # else:
    #     masks = torch.zeros((0, height, width), dtype=torch.uint8)
    return masks

In [12]:
def calculate_IoU(pred, target):
    '''
    Calculates the Intersection over Union(Intersection over Union).
    '''
    intersection = np.logical_and(target, pred)
    union = np.logical_or(target, pred)
    iou_score = np.sum(intersection) / np.sum(union)

    return iou_score

In [13]:
# with open('predictions_toy.json') as f:
#     preds = json.load(f)
# video28_objects = [p for p in preds if p["video_id"] == 28]
# video28_objects_dict = {i:obj for i, obj in enumerate(video28_objects)}

In [14]:
import json
with open('predictions_model_perceptual_gan_v2_7x7_003.json') as f:
    preds = json.load(f)
    
# with open('predictions_toy.json') as f:
#     preds = json.load(f)
annotation_indices = [28, 33, 42, 59]

In [15]:
len(preds)

8782

In [50]:
threshold = 0.1
import copy

last_preds_list = []
counter = 0
for ann_idx in annotation_indices:
    video_objects = [p for p in preds if p["video_id"] == ann_idx]
    video_objects_dict = {i:obj for i, obj in enumerate(video_objects)}
    
    for i in range(9, 350, 10):

    #     candidate_objs = [(idx, obj) for (idx, obj) in enumerate(video_objects) if obj["segmentations"][i] != None]
        candidate_objs = [(idx, obj) for idx, obj in video_objects_dict.items() if obj["segmentations"][i] != None]
    #     objs_to_concetenate = [(idx, obj) for (idx, obj) in enumerate(video_objects) if obj["segmentations"][i+1] != None]
        objs_to_concetenate = [(idx, obj) for idx, obj in video_objects_dict.items() if obj["segmentations"][i+1] != None]

        keys_to_remove_list = []
        for cand_idx, cand in candidate_objs:
            iou_calcs = []

            for next_seq_obj_idx, next_seq_obj in objs_to_concetenate:
                a = convert_coco_poly_to_mask([cand["segmentations"][i]], 1080, 1080)[0]
                b = convert_coco_poly_to_mask([next_seq_obj["segmentations"][i+1]], 1080, 1080)[0]
                iou_calc = calculate_IoU(a, b)
                iou_calcs.append(iou_calc)

            if max(iou_calcs) > threshold:
                counter += 1
                max_index = np.argmax(iou_calcs)

                cand["segmentations"][i+1:i+11] = copy.deepcopy(objs_to_concetenate[max_index][1]["segmentations"][i+1:i+11])
                video_objects_dict[cand_idx]["segmentations"] = copy.deepcopy(cand["segmentations"])
                keys_to_remove_list.append(objs_to_concetenate[max_index][0])
    #             print(f"cand_idx:{cand_idx} concetanated with next_seq_obj_idx: {i+1+max_index}, with iou: {max(iou_calcs)}")

            else:
                max_index = np.argmax(iou_calcs)
    #             print(f"cand_idx:{cand_idx} CANNOT BE next_seq_obj_idx: {i+1+max_index}, max iou: {max(iou_calcs)}")

        keys_to_remove_list = list(set(keys_to_remove_list))
        for key in keys_to_remove_list:
            video_objects_dict.pop(key)

    last_preds_list.append(copy.deepcopy(video_objects_dict))
    print(f"{ann_idx} done.")

<ipython-input-45-b3e3190e6bba>:7: RuntimeWarning: invalid value encountered in long_scalars
  iou_score = np.sum(intersection) / np.sum(union)


28 done.
33 done.
42 done.
59 done.


In [51]:
4

4

In [53]:
a = [list(vid.values()) for vid in last_preds_list]

import itertools
all_preds = list(itertools.chain.from_iterable(a))

In [55]:
import json
with open('evaluation/predictions_model_perceptual_gan_v2_003_last_version.json', 'w') as f:
    json.dump(all_preds, f)

In [52]:
for i in last_preds_list:
    print(len(i))

1201
1187
1279
1614


In [12]:
video28_objects = [p for p in preds if p["video_id"] == 28]
# video28_objects_dict = {i:obj for i, obj in enumerate(video28_objects)}
print(len(video28_objects))

video28_objects = [p for p in preds if p["video_id"] == 33]
# video28_objects_dict = {i:obj for i, obj in enumerate(video28_objects)}
print(len(video28_objects))

video28_objects = [p for p in preds if p["video_id"] == 42]
# video28_objects_dict = {i:obj for i, obj in enumerate(video28_objects)}
print(len(video28_objects))

video28_objects = [p for p in preds if p["video_id"] == 59]
# video28_objects_dict = {i:obj for i, obj in enumerate(video28_objects)}
print(len(video28_objects))

1356
832
622
405


In [ ]:
a = [list(vid.values()) for vid in last_preds_list]

import itertools
all_preds = list(itertools.chain.from_iterable(a))

In [64]:
# preds_list = [vid for vid in last_preds_list obj for obj in list(vid.values())]

SyntaxError: invalid syntax (<ipython-input-64-bf5236dd0e9a>, line 1)

In [13]:
video28_objects = [p for p in preds if p["video_id"] == 28]
# video28_objects_dict = {i:obj for i, obj in enumerate(video28_objects)}
len(video28_objects)

1425

In [ ]:
gt_data_path = "evaluation/gt_data_eval.json"
with open(gt_data_path) as fp:
    gt_data = json.load(fp)

In [ ]:
gt_data_33 = [ann for ann in gt_data["annotations"] if ann["video_id"] == 59]

In [ ]:
len(gt_data_33)

In [ ]:
len(gt_data["annotations"][0]["segmentations"])

In [ ]:
gt_data["annotations"][0]["video_id"]

In [ ]:
annotation_indices = [28, 33, 42, 59]

In [ ]:
for i in last_preds_list:
    print(len(i))

In [ ]:
len(last_preds_list[0])

In [ ]:
for i in annotation_indices:
    print(len([p for p in preds if p["video_id"] == i]))

In [ ]:
last_preds_list[0][2]

In [ ]:
with open('predictions_model_gradient_2.json') as f:
    preds = json.load(f)
len(preds)

In [ ]:
with open('predictions_toy.json') as f:
    preds = json.load(f)
    
len(preds)

In [ ]:
preds[0]["segmentations"]

In [ ]:
preds[0].keys()

In [73]:
len(all_preds)

1958

In [74]:
len(preds)

3341

In [75]:
import json
with open('evaluation/all_preds.json', 'w') as f:
    json.dump(all_preds, f)

In [ ]:
gt_data_path = "evaluation/gt_data_eval.json"
with open(gt_data_path) as fp:
    example_data = json.load(fp)

In [76]:
gt_data_path = "evaluation/gt_data_eval.json"
with open(gt_data_path) as fp:
    gt_data = json.load(fp)